# Project 2 - Medical Imaging 

## Imports

In [122]:
import os 
import re 

import numpy as np 
import pandas as pd 
from PIL import Image 
from matplotlib import pyplot as plt 
import seaborn as sns 

from skimage import morphology 
from skimage.transform import rotate 


from time import time 

## Global Variables

In [123]:
# Initial parameters for running the notebook 
PREPROCESS = False
COMPUTE_FEATURES = True 

In [124]:
# Paths to directories used 

ROOT_DIR = "../"
DATA_DIR = ROOT_DIR + "data"
IM_DIR = DATA_DIR + "/ISIC-2017_Training_Data"
MASK_DIR = DATA_DIR + "/ISIC-2017_Training_Part1_GroundTruth"
DIAGNOSIS_PATH = DATA_DIR + "/ISIC-2017_Training_Part3_GroundTruth.csv"


In [125]:
# Load all filenames into a dictionary
FILENAMES = {}
FILENAMES['images'] = sorted([IM_DIR + "/" + i for i in list(os.walk(IM_DIR))[0][2]])
FILENAMES['masks'] = sorted([f"{MASK_DIR}/{i}" for i in list(os.walk(MASK_DIR))[0][2]])
FILENAMES['image_num'] = len(FILENAMES['images'])

## Helper functions 

In [126]:
def make_sides_even(image): 
    '''
    Function to make the numbers of columns and rows in 
    an image even. 
    Input: An image 
    Output: An image
    '''
    # Convert image to numpy array
    image = np.array(image)
    
    # Check if the number of rows is even 
    if image.shape[0] % 2 != 0: 
        # Delete the first row
        image = np.delete(image,0,axis = 0)
    
    # Check if the number of columns is even 
    if image.shape[1] % 2 != 0: 
        # Delete the first column
        image = np.delete(image,0,axis = 1)
    
    # Convert numpy array back to image 
    image = Image.fromarray(image)
    # Return the updated image 
    return image

In [127]:
def filter_and_crop_image(image_path,mask_path): 
    '''
    Function to filter filter an image based on a given mask 
    and crop both image and mask to the relevant area. 
    '''
    # Instantiate both image and mask 
    image = Image.open(image_path) 
    mask = Image.open(mask_path)
    
    # Crop both image and mask to the bounding box
    # of the mask. 
    image_crop = image.crop(mask.getbbox())
    mask_crop = mask.crop(mask.getbbox())
    
    # Make the length and height of the the image and 
    # mask even 
    image_crop = make_sides_even(image_crop)
    mask_crop = make_sides_even(mask_crop)

    # Instantiate a blank image for a composite image
    tmp_image = Image.new("RGB",image_crop.size, 0)

    # Create a composite image based on the blank image, 
    # the cropped image and the mask 
    filtered_image = Image.composite(image_crop,tmp_image,mask_crop)
    
    # Return the filtered image, the cropped mask and the cropped image 
    return filtered_image, mask_crop, image_crop

## Preprocessing

In [128]:
# Check if the PREPROCESS variable is True 
if PREPROCESS: 
    # Try to create directories for the filtered images, cropped masks 
    # and cropped images 
    try: 
        os.makedirs(DATA_DIR + "/filtered_images")
        os.makedirs(DATA_DIR + "/filtered_masks")
        os.makedirs(DATA_DIR + "/cropped_images")

    except FileExistsError:
        print("Directories exist")
    except IsADirectoryError: 
        print("Directories exist")

    # Instantiate variables containing the paths to the filtered images, 
    # cropped masks and cropped images 
    IMAGE_FILTER_DIR = DATA_DIR + "/filtered_images"
    MASK_FILTER_DIR = DATA_DIR + "/filtered_masks"
    IMAGE_CROP_DIR = DATA_DIR + "/cropped_images"

    # Loop over all images and masks based on the number of images
    for i in range(FILENAMES['image_num']):
        # Instantiate variables with the paths to the currently processing image
        # and mask 
        mask_path = FILENAMES['masks'][i]
        image_path = FILENAMES['images'][i]

        # Filter and crop the image and mask 
        image, mask, crop = filter_and_crop_image(image_path,mask_path)

        # Instantiate variables containing the image and mask name 
        image_name = image_path.split("/")[-1].split(".")[-2]
        mask_name = mask_path.split("/")[-1].split(".")[-2]

        # Save the generated images and mask to files 
        image.save(IMAGE_FILTER_DIR + "/" + image_name + ".jpg")
        mask.save(MASK_FILTER_DIR + "/" + mask_name + ".png")
        crop.save(IMAGE_CROP_DIR + "/" + image_name + ".jpg")

        # Clear the temporary variables 
        del mask_path, image_path, image, mask, crop, image_name, mask_name
    
    # Load all the new filenames into the FILENAMES dictionary
    FILENAMES['filtered_images'] = sorted([f"{IMAGE_FILTER_DIR}/{i}" for i in list(os.walk(IMAGE_FILTER_DIR))[0][2]])
    FILENAMES['cropped_images'] = sorted([f"{IMAGE_CROP_DIR}/{i}" for i in list(os.walk(IMAGE_CROP_DIR))[0][2]])
    FILENAMES['cropped_masks'] = sorted([f"{MASK_FILTER_DIR}/{i}" for i in list(os.walk(MASK_FILTER_DIR))[0][2]])
else: 
    # Instantiate variables containing the paths to the filtered images, 
    # cropped masks and cropped images 
    IMAGE_FILTER_DIR = DATA_DIR + "/filtered_images"
    MASK_FILTER_DIR = DATA_DIR + "/filtered_masks"
    IMAGE_CROP_DIR = DATA_DIR + "/cropped_images"

    # Load all the new filenames into the FILENAMES dictionary
    FILENAMES['filtered_images'] = sorted([f"{IMAGE_FILTER_DIR}/{i}" for i in list(os.walk(IMAGE_FILTER_DIR))[0][2]])
    FILENAMES['cropped_images'] = sorted([f"{IMAGE_CROP_DIR}/{i}" for i in list(os.walk(IMAGE_CROP_DIR))[0][2]])
    FILENAMES['cropped_masks'] = sorted([f"{MASK_FILTER_DIR}/{i}" for i in list(os.walk(MASK_FILTER_DIR))[0][2]])


## Feature Extraction Functions

In [129]:
## Find the area and perimeter of the mask 
def get_area_perimeter(mask): 
    '''
    Function which takes in a mask for a
    given image and returns the area and 
    perimeter of the mask. 
    '''

    # Convert the mask to a Numpy array containing 1's and 0's
    mask = np.where(np.array(mask) ==255,1,0)

    # Calculate the area of the mask as the sum of the Numpy array
    area = np.sum(mask)
    
    # Erode the edge of the mask in order to calculate the perimeter 
    perimeters = []
    for i in range(1,5):
        mask_erosion = morphology.binary_erosion(mask,morphology.disk(i))
        
        # Calculate the perimeter 
        perimeter = np.sum(mask - mask_erosion)
        perimeters.append(perimeter)
    # Return the area and the perimeter 
    return area, perimeters

In [130]:
def get_compactness(mask): 
    '''
    Function which takes in a mask for a given 
    image, calls the get_area_perimeter function
    to get the area and perimeter, and returns a 
    compactness score based upon [CITATION NEEDED]. 
    '''
    # Calculate the area and the perimeter 
    area, perimeters = get_area_perimeter(mask) 
    compactness_list = []
    for i in perimeters: 

    # Calculate the compactness 
        compactness = i ** 2 / (4 * np.pi * area)
        compactness_list.append(compactness)
    # Return the compactness 
    compactness = compactness_list 
    return compactness 

In [131]:
def get_asymmetry(mask, rotation=45): 
    '''
    Takes in a mask for a given image, rotates it 
    180 times by one degree, compares the left and 
    right half and returns an average asymmetry score.
    '''
    # Instantiate the mask as a Numpy array 
    mask = np.array(mask)
    axes = 0 
    # Get the length and width of the mask 
    length, width = mask.shape 
    # Define a number of pixels for padding to avoid information loss
    # pad_size = int(max((length, width))/)
    
    # Redefine the length to include the padded pixels 
    # length = length + 2 * pad_size 
    # Pad the mask 
    # mask = np.pad(mask,pad_size)
    
    # Instantiate a list to holds the calculated differences  
    diffs = []

    while axes * rotation < 180:
        temp_mask = rotate(mask, axes * rotation)
        # length_lesion = np.nonzero(np.sum(temp_mask, axis = 0))[0][-1] - np.nonzero(np.sum(temp_mask, axis = 0))[0][0]

        left_mask = temp_mask[:, :int(length/2)] 
        right_mask = temp_mask[:, int(length/2):]
        right_mask = np.fliplr(right_mask)
        rotation_diff = np.where(left_mask != right_mask)
    
        diffs.append(np.sum(rotation_diff))
        
        axes += 1
    
    diff = np.mean(diffs)
    
    return diff



In [132]:
def get_average_luminance(image): 
    '''
    A function which takes in an image and
    returns the average luminance. 
    Input: A PIL Image 
    Output: Average luminance of the image
    '''
    # Convert the image to grayscale and then to a Numpy array 
    grayscale = np.array(image.convert('L'))

    # Calculate the mean of the luminance
    average_luminance = round(np.mean(grayscale[grayscale > 0]))

    # Return the average luminance 
    return average_luminance

In [133]:
def get_luminance_variability(image,measure="variance"): 

    # Convert the image to grayscale and then to a Numpy array 
    grayscale = np.array(image.convert('L'))

    # Check if measure is 'variance'
    if measure == 'variance': 
        # Using numpy's variance method, return the variance in luminance
        return round(np.var(grayscale[grayscale > 0]))
    # Check if measure is 'standard_deviation'
    elif measure == "standard_deviation": 
        # Using numpy's standard deviation method, return the standard deviation
        return round(np.std(grayscale[grayscale > 0]))
    else: 
        # If measure is neither 'variance' nor 'standard_deviation', raise a 
        # ValueError. 
        raise ValueError("Only 'variance' or 'standard_deviation' accepted.") 

In [134]:
def get_avg_color(image): 
    '''
    A function which takes in an image and returns 
    the average color of the image. 
    '''
    # Split the image into separate color channels 
    r, g, b = image.split()
    # Instantiate a numpy array based on each color channel 
    r = np.array(r)
    g = np.array(g)
    b = np.array(b)
    # Calculate the mean of each color channel 
    average_color = (
        round(np.mean(r[r > 0])), 
        round(np.mean(g[g > 0])), 
        round(np.mean(b[b > 0]))
    )
    # Return a tuple of the average color 
    return average_color

In [135]:
def get_color_variance(image,measure="variance"): 
    '''
    A function which takes in an image and 
    returns the variance of the color. 
    '''
    # Split the image into separate color channels 
    r, g, b = image.split()
    # Instantiate a numpy array based on each color channel 
    r = np.array(r)
    g = np.array(g)
    b = np.array(b)
    # Check if measure is 'variance'
    if measure == "variance": 
        # Using numpy's variance method, calculate the variance of each color
        rgb = (
            np.var(r[r>0]),
            np.var(g[g>0]),
            np.var(b[b>0])
        )
    # Check if measure is 'standard_deviation'
    elif measure == "standard_deviation": 
        # Using numpy's standard deviation method, calculate the standard deviation
        # of each color 
        rgb = (
            np.std(r[r>0]),
            np.std(g[g>0]),
            np.std(b[b>0])
        )
    else: 
        # If measure is neither 'variance' nor 'standard_deviation', raise 
        # a ValueError 
        raise ValueError("Only 'variance' or 'standard_deviation' accepted.") 
    # Return the mean of the variances or standard deviation of the 3 colors
    return np.mean(rgb)

## Feature Extraction 

In [137]:
# Check if the COMPUTE_FEATURES variable is True 
if COMPUTE_FEATURES: 
    # Instantiate a dictionary to store the features and image_id's 
    # feature_dictionary = {
    #     "image_id": [],
    #     "area": [], 
    #     "perimeter": [],
    #     "compactness": [], 
    #     "asymmetry": [], 
    #     "luminance_average": [],
    #     "luminance_variance": [],
    #     "red_average": [],
    #     "green_average": [],
    #     "blue_average": [],
    #     "color_variance": [],
    # }
    feature_dictionary = {
        "image_id": [],
        "area": [], 
        "perimeter_1": [],
        "perimeter_2": [],
        "perimeter_3": [],
        "perimeter_4": [],
        
        "compactness_1": [],
        "compactness_2": [],
        "compactness_3": [],
        "compactness_4": [], 
        "asymmetry": [], 
        "luminance_average": [],
        "luminance_variance": [],
        "red_average": [],
        "green_average": [],
        "blue_average": [],
        "color_variance": [],
    }

    # Loop over all filtered images and cropped masks 
    for i in range(FILENAMES['image_num']): 
        # Instantiate variables containing the paths to the image and mask
        filtered_image_path = FILENAMES['filtered_images'][i]
        cropped_mask_path = FILENAMES['cropped_masks'][i]

        # Get the image name from the image path 
        image_name = filtered_image_path.split("/")[-1].split(".")[-2]

        print(f"Currently working on {i} - Image id: {image_name}")

        # Open both the image and mask as PIL Images 
        filtered_image = Image.open(filtered_image_path)
        cropped_mask = Image.open(cropped_mask_path)

        # Calculate all features and append them to the relevant list in 
        # the feature_dictionary 
        feature_dictionary['image_id'].append(image_name)
        area, perimeter = get_area_perimeter(cropped_mask)
        feature_dictionary['area'].append(area)
        
        feature_dictionary['perimeter_1'].append(perimeter[0])
        feature_dictionary['perimeter_2'].append(perimeter[1])
        feature_dictionary['perimeter_3'].append(perimeter[2])
        feature_dictionary['perimeter_4'].append(perimeter[3])
        compactness = get_compactness(cropped_mask)
        feature_dictionary['compactness_1'].append(compactness[0])
        feature_dictionary['compactness_2'].append(compactness[1])
        feature_dictionary['compactness_3'].append(compactness[2])
        feature_dictionary['compactness_4'].append(compactness[3])


        # feature_dictionary['compactness'].append(get_compactness(cropped_mask))

    
        feature_dictionary['asymmetry'].append(get_asymmetry(cropped_mask))
        feature_dictionary['luminance_average'].append(get_average_luminance(filtered_image))
        feature_dictionary['luminance_variance'].append(get_luminance_variability(filtered_image))
        red, green, blue = get_avg_color(filtered_image)
        feature_dictionary['red_average'].append(red)
        feature_dictionary['green_average'].append(green)
        feature_dictionary['blue_average'].append(blue)
        feature_dictionary['color_variance'].append(get_color_variance(filtered_image))
    # Instantiate a pandas DataFrame based on the feature_dictionary
    features = pd.DataFrame(feature_dictionary)
    # Write the features DataFrame to .csv 
    features.to_csv(ROOT_DIR + "/features/feature_set.csv",sep=";",index=False)
else: 
    # Read the previously calculated feature set into a pandas DataFrame 
    features = pd.read_csv(ROOT_DIR + "/features/feature_set.csv", sep=";", index_col=False)

    

Currently working on 0 - Image id: ISIC_0000000
Currently working on 1 - Image id: ISIC_0000001
Currently working on 2 - Image id: ISIC_0000002
Currently working on 3 - Image id: ISIC_0000003
Currently working on 4 - Image id: ISIC_0000004
Currently working on 5 - Image id: ISIC_0000006
Currently working on 6 - Image id: ISIC_0000007
Currently working on 7 - Image id: ISIC_0000008
Currently working on 8 - Image id: ISIC_0000009
Currently working on 9 - Image id: ISIC_0000010
Currently working on 10 - Image id: ISIC_0000011
Currently working on 11 - Image id: ISIC_0000012
Currently working on 12 - Image id: ISIC_0000013
Currently working on 13 - Image id: ISIC_0000014
Currently working on 14 - Image id: ISIC_0000015
Currently working on 15 - Image id: ISIC_0000016
Currently working on 16 - Image id: ISIC_0000017
Currently working on 17 - Image id: ISIC_0000018
Currently working on 18 - Image id: ISIC_0000019
Currently working on 19 - Image id: ISIC_0000020
Currently working on 20 - Imag

In [171]:
feature_scales = pd.DataFrame([features['perimeter_1'],
                                features['perimeter_2'],
                                features['perimeter_3'],
                                features['perimeter_4'],
                                features['compactness_1'],
                                features['compactness_2'],
                                features['compactness_3'],
                                features['compactness_4']])


In [172]:
feature_scales = feature_scales.transpose()
feature_scales.describe()

,perimeter_1,perimeter_2,perimeter_3,perimeter_4,compactness_1,compactness_2,compactness_3,compactness_4
count,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,3304.243000,6534.197000,10053.41300,13237.224500,1.351726,5.161827,12.291431,21.054818
std,3549.302668,6886.838111,10335.74022,13553.218592,1.676975,5.270066,11.407258,17.767377
min,317.000000,632.000000,1007.00000,1317.000000,0.088357,0.354235,0.875832,1.527257
25%,1434.000000,2852.000000,4488.50000,5928.750000,0.833657,3.333039,8.189498,14.247109
50%,2143.000000,4260.500000,6622.50000,8744.000000,1.040209,4.147590,10.036135,17.546806
75%,3594.500000,7159.500000,11179.75000,14684.750000,1.391115,5.498148,13.155825,22.918351
max,35203.000000,65235.000000,98278.00000,123997.000000,35.137359,104.152494,216.949080,330.227809


In [173]:
feature_scales['p_1_2'] = feature_scales['perimeter_2'] / feature_scales['perimeter_1']
feature_scales['p_2_3'] = feature_scales['perimeter_3'] / feature_scales['perimeter_2']
feature_scales['p_3_4'] = feature_scales['perimeter_4'] / feature_scales['perimeter_3']
feature_scales['p_1_3'] = feature_scales['perimeter_3'] / feature_scales['perimeter_1']
feature_scales['p_2_4'] = feature_scales['perimeter_4'] / feature_scales['perimeter_2']
feature_scales['p_1_4'] = feature_scales['perimeter_4'] / feature_scales['perimeter_1']


feature_scales['c_1_2'] = feature_scales['compactness_2'] / feature_scales['compactness_1']
feature_scales['c_2_3'] = feature_scales['compactness_3'] / feature_scales['compactness_2']
feature_scales['c_3_4'] = feature_scales['compactness_4'] / feature_scales['compactness_3']
feature_scales['c_1_3'] = feature_scales['compactness_3'] / feature_scales['compactness_1']
feature_scales['c_1_4'] = feature_scales['compactness_4'] / feature_scales['compactness_1']
feature_scales['c_2_4'] = feature_scales['compactness_4'] / feature_scales['compactness_2']


In [174]:
tmp_df = feature_scales.drop(['perimeter_2','perimeter_1','perimeter_3','perimeter_4','compactness_1','compactness_2','compactness_3','compactness_4'],axis=1)
tmp_df.describe()

,p_1_2,p_2_3,p_3_4,p_1_3,p_2_4,p_1_4,c_1_2,c_2_3,c_3_4,c_1_3,c_1_4,c_2_4
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,1.987185,1.553781,1.317382,3.088323,2.046744,4.068837,3.950455,2.416411,1.735724,9.553750,16.586499,4.192677
std,0.039361,0.046649,0.015102,0.126566,0.059321,0.176303,0.148953,0.144923,0.039184,0.759123,1.350396,0.238885
min,1.707717,1.422143,1.227119,2.428617,1.745811,2.981350,2.916297,2.022490,1.505821,5.898182,8.888451,3.047855
25%,1.989758,1.499762,1.311703,2.987230,1.994772,3.974378,3.959135,2.249287,1.720566,8.923541,15.795684,3.979115
50%,1.993877,1.563194,1.319376,3.123926,2.062527,4.121043,3.975544,2.443575,1.740754,9.758913,16.982995,4.254020
75%,2.000692,1.591553,1.328562,3.175755,2.089428,4.172067,4.002767,2.533040,1.765076,10.085417,17.406140,4.365709
max,2.008152,1.688544,1.339787,3.375522,2.188776,4.375522,4.032675,2.851179,1.795029,11.394146,19.145189,4.790738
